In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import cv2
from PIL import Image
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc, roc_curve
from tqdm import tqdm

import time

In [7]:
# Keras & Tensorflow
import keras
import tensorflow as tf
from keras import layers
from keras.models import Model
from keras.layers import Lambda, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D , Conv2D , MaxPooling2D
from keras.layers import  Dropout , BatchNormalization , Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from keras.callbacks import Callback , ReduceLROnPlateau , ModelCheckpoint
from sklearn.metrics import cohen_kappa_score, accuracy_score
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy

In [8]:
benign_train = np.load('/content/drive/MyDrive/CSE499/numpy/benign_train.npy')
malign_train = np.load('/content/drive/MyDrive/CSE499/numpy/malign_train.npy')
benign_test = np.load('/content/drive/MyDrive/CSE499/numpy/benign_test.npy')
malign_test = np.load('/content/drive/MyDrive/CSE499/numpy/malign_test.npy')
print('Done Loaded :)')

Done Loaded :)


In [9]:
#Shape of our dataset
print(f'Benign Train:',benign_train.shape)
print(f'Malignant Train',malign_train.shape)
print(f'Benign Test:',benign_test.shape)
print(f'Malignant Test',malign_test.shape)

Benign Train: (3500, 224, 224, 3)
Malignant Train (3496, 224, 224, 3)
Benign Test: (1500, 224, 224, 3)
Malignant Test (1500, 224, 224, 3)


In [10]:
benign_train_label = np.zeros(len(benign_train),dtype=float)
malign_train_label = np.ones(len(malign_train),dtype=float)
benign_test_label = np.zeros(len(benign_test),dtype=float)
malign_test_label = np.ones(len(malign_test),dtype=float) 

X_train = np.concatenate((benign_train, malign_train), axis = 0)
Y_train = np.concatenate((benign_train_label, malign_train_label), axis = 0)
X_test = np.concatenate((benign_test, malign_test), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

Y_train = to_categorical(Y_train, num_classes= 2)
Y_test = to_categorical(Y_test, num_classes= 2)

In [11]:
print(f'X train shape:',X_train.shape)
print(f'X test shape:',X_test.shape)
print(f'Y train shape:',Y_train.shape)
print(f'Y test shape:',Y_test.shape)

X train shape: (6996, 224, 224, 3)
X test shape: (3000, 224, 224, 3)
Y train shape: (6996, 2)
Y test shape: (3000, 2)


In [12]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=10)

In [13]:
print(f'x train shape:',x_train.shape)
print(f'x val shape:',x_val.shape)
print(f'y train shape:',y_train.shape)
print(f'y val shape:',y_val.shape)

x train shape: (5596, 224, 224, 3)
x val shape: (1400, 224, 224, 3)
y train shape: (5596, 2)
y val shape: (1400, 2)


In [14]:
from tensorflow.keras.applications import ResNet101

def res_model101(backbone , lr = 1e-4):
  resmodel101 = Sequential()
  resmodel101.add(backbone)
  resmodel101.add(layers.GlobalAveragePooling2D())
  resmodel101.add(layers.Dropout(0.5))
  resmodel101.add(layers.BatchNormalization())
  resmodel101.add(layers.Dense(2 , activation='softmax'))
  
  resmodel101.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=lr),
        metrics=['accuracy']
    )
    
  return resmodel101


resenet101 = ResNet101(
    weights = 'imagenet',
    include_top = False,
    input_shape =(224 , 224 , 3)
)
#call the model 
resmodel101 = res_model101(resenet101 , lr = 1e-4)
resmodel101.build((None, 224, 224, 3))
resmodel101.summary()

171458560/171446536 [==============================] - 13s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                      

In [15]:
# Training the teacher model with Resnet50
# Train the teacher model as usual

epochs = 20
batch_size = 64
resmodel101_student_his = resmodel101.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val))

Epoch 1/20
88/88 [==============================] - 396s 3s/step - loss: 0.1844 - accuracy: 0.9317 - val_loss: 0.1446 - val_accuracy: 0.9443
Epoch 2/20
88/88 [==============================] - 245s 3s/step - loss: 0.0700 - accuracy: 0.9739 - val_loss: 0.1290 - val_accuracy: 0.9579
Epoch 3/20
88/88 [==============================] - 249s 3s/step - loss: 0.0386 - accuracy: 0.9843 - val_loss: 0.1326 - val_accuracy: 0.9607
Epoch 4/20
88/88 [==============================] - 246s 3s/step - loss: 0.0275 - accuracy: 0.9905 - val_loss: 0.1450 - val_accuracy: 0.9593
Epoch 5/20
88/88 [==============================] - 239s 3s/step - loss: 0.0161 - accuracy: 0.9945 - val_loss: 0.1342 - val_accuracy: 0.9621
Epoch 6/20
88/88 [==============================] - 239s 3s/step - loss: 0.0149 - accuracy: 0.9950 - val_loss: 0.4126 - val_accuracy: 0.9486
Epoch 7/20
88/88 [==============================] - 235s 3s/step - loss: 0.0166 - accuracy: 0.9954 - val_loss: 0.1666 - val_accuracy: 0.9543
Epoch 8/20
88

In [16]:
Y_val_pred = resmodel101.predict(x_val)
print(f'The ResNet101 model Accuracy on the Validation Set:',accuracy_score(np.argmax(y_val, axis=1), np.argmax(Y_val_pred, axis=1)))

The ResNet101 model Accuracy on the Validation Set: 0.9678571428571429


In [17]:
#Now let's check my Y_test values
print(f'My Y_test values are:\n' ,Y_test)

My Y_test values are:
 [[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [19]:
#Now let's check my predcited values from X_test dataset	
import timeit

start = timeit.default_timer()
#Your statements here
y_pred = resmodel101.predict(X_test)
print(f'My predicted Y_test values are:\n' ,y_pred)

stop = timeit.default_timer()
print('\nTime: ',stop - start,'sec')

My predicted Y_test values are:
 [[9.9999976e-01 2.3670326e-07]
 [9.9999785e-01 2.1068079e-06]
 [1.6881677e-07 9.9999988e-01]
 ...
 [1.0795724e-02 9.8920435e-01]
 [6.7005067e-07 9.9999928e-01]
 [9.9999988e-01 7.4649392e-08]]

Time:  52.32076698800029 sec


In [20]:
#Now let's check the accuracy between the original & predicted (Y_test , y_pred)
print(f'My accuracy on Student model with ResNet101 on the Test set is:',accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1)))

My accuracy on Student model with ResNet101 on the Test set is: 0.9983333333333333


In [21]:
# Print the classification report
print(classification_report( np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1500
           1       1.00      1.00      1.00      1500

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

